In [1]:
# W&B Miner Performance Analysis

import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from data import (
    get_wandb_runs,
    get_unique_validator_run_names,
    compute_miner_performance,
    download_challenge_media,
    merge_performance_and_downloads
)



In [11]:
# Set NETWORK for `project` to be set accordingly 
NETWORK = 'finney'  # or 'test'

entity = "bitmindai"
project = "bitmind-subnet" if NETWORK == 'finney' else 'bitmind'

# Time range settings
start_date = datetime.datetime(2025, 2, 24)  # Change to your desired start date
end_date = datetime.datetime(2025, 2, 25)    # Change to your desired end date

# Filtering settings
validator_run_name = 'validator-236-2.2.0'  # Optionally specify a run name
miner_uids = [33, 64]                       # Optionally specify a specific miner UID


## Get W&B Runs

In [12]:
# Get all W&B runs within the specified time range
print(f"Fetching W&B runs from {entity}/{project} between {start_date.date()} and {end_date.date()}")
runs = get_wandb_runs(
    entity=entity,
    project=project,
    start_ts=start_date,
    end_ts=end_date,
    validator_run_name=validator_run_name
)

print(f"Found {len(runs)} runs")

# If you're curious about available run names
run_names = get_unique_validator_run_names(
    entity=entity,
    project=project,
    start_ts=start_date,
    end_ts=end_date
)

print(f"Available run names: {run_names}")


Fetching W&B runs from bitmindai/bitmind-subnet between 2025-02-24 and 2025-02-25
Querying W&B with filters: {'display_name': 'validator-236-2.2.0', 'created_at': {'$gte': '2025-02-24T00:00:00', '$lte': '2025-02-25T00:00:00'}}
Found 9 runs
Querying bitmindai/bitmind-subnet with filters: {'created_at': {'$gte': '2025-02-24T00:00:00', '$lte': '2025-02-25T00:00:00'}}
Available run names: {'validator-160-2.2.0', 'data-generator-19-2.2.0', 'cache-updater-19-2.2.0', 'validator-254-2.2.0', 'validator-140-2.2.0', 'validator-135-2.2.0', 'data-generator-135-2.2.0', 'cache-updater-166-2.2.0', 'data-generator-236-2.2.0', 'cache-updater-236-2.2.0', 'data-generator-166-2.2.0', 'validator-6-2.2.0', 'data-generator-254-2.2.0', 'cache-updater-160-2.2.0', 'data-generator-140-2.2.0', 'cache-updater-254-2.2.0', 'validator-236-2.2.0', 'validator-19-2.2.0', 'cache-updater-135-2.2.0', 'data-generator-160-2.2.0', 'data-generator-6-2.2.0', 'validator-166-2.2.0', 'cache-updater-6-2.2.0', 'cache-updater-140-2.2.

## Miner Performance Metrics

In [13]:
print("Computing miner performance metrics...")
perf_results = compute_miner_performance(
    wandb_validator_runs=runs,
    miner_uids=miner_uids,
    start_ts=start_date,
    end_ts=end_date,
    validator_run_name=validator_run_name
)
predictions_df = perf_results['predictions']
performance_df = perf_results['performance']
print("Done.")

Computing miner performance metrics...
Done.


In [14]:
# Display a sample of the predictions
print("\nPredictions sample:")
display(predictions_df.head())

# Display the performance metrics
print("\nPer-miner performance metrics:")
display(performance_df.head())



Predictions sample:


,modality,uid,prediction,label,wandb_filepath,validator_run,timestamp
0,video,64,"[0, 1, 0]",1,No Media Found,validator-236-2.2.0,1.740384e+09
1,video,33,"[0.9990643605240621, 0.0009356394759379328, 0]",0,media/videos/video_121_e0858f29644baad423da.gif,validator-236-2.2.0,1.740385e+09
2,video,33,"[0.9124413390178233, 0.08755866098217666, 0]",0,media/videos/video_124_f9a1d871060237216156.gif,validator-236-2.2.0,1.740385e+09
3,video,64,"[0, 1, 0]",1,media/videos/video_128_2dcdc5b28395c9cc09b3.gif,validator-236-2.2.0,1.740385e+09
4,image,33,"[0.3816100060939789, 0.5700175762176514, 0.048...",0,media/images/image_129_1586010fb1c66f0bb4d2.png,validator-236-2.2.0,1.740385e+09



Per-miner performance metrics:


,multiclass_accuracy,multiclass_precision,multiclass_recall,multiclass_f1,multiclass_mcc,binary_accuracy,binary_precision,binary_recall,binary_f1,binary_auc,binary_mcc,sample_size,uid,modality
0,0.960526,0.961388,0.960526,0.960091,0.932442,0.960526,0.957447,0.978261,0.967742,0.997101,0.917262,76,33,image
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,56,33,video
2,0.972603,0.973045,0.972603,0.972619,0.953060,0.986301,0.973684,1.000000,0.986667,0.986111,0.972953,73,64,image
3,0.956522,0.957116,0.956522,0.956615,0.911596,0.956522,0.974359,0.950000,0.962025,0.957759,0.911596,69,64,video


## Optionally Download Media

In [15]:
# Download settings
download_media = True      # Set to False to skip media downloads
download_dir = "./downloads"
download_images = True
download_videos = True
download_limit = 10        # Limit to 10 files (change as needed)

# Create downloads directory if needed
if download_media and not os.path.exists(download_dir):
    os.makedirs(download_dir)



In [16]:
download_df = None
verbose = False

if download_media:
    print(f"\nDownloading media files to {download_dir}...")
    
    download_df = download_challenge_media(
        wandb_validator_runs=runs,
        download_dest=download_dir,
        download_images=download_images,
        download_videos=download_videos,
        download_limit=download_limit,
        miner_uids=miner_uids,
        start_ts=start_date,
        end_ts=end_date,
        validator_run_name=validator_run_name,
        verbose=verbose
    )
    
    print(f"Downloaded media info:")
    display(download_df.head())
    
    # Count media types
    media_counts = download_df['modality'].value_counts()
    print(f"\nMedia counts by type:")
    display(media_counts)


Downloaded media info:


,validator_run,modality,uid,wandb_filepath,local_filepath,timestamp
0,validator-236-2.2.0,video,33,media/videos/video_121_e0858f29644baad423da.gif,./downloads/media/videos/video_121_e0858f29644...,1.740385e+09
1,validator-236-2.2.0,video,33,media/videos/video_124_f9a1d871060237216156.gif,./downloads/media/videos/video_124_f9a1d871060...,1.740385e+09
2,validator-236-2.2.0,video,64,media/videos/video_128_2dcdc5b28395c9cc09b3.gif,./downloads/media/videos/video_128_2dcdc5b2839...,1.740385e+09
3,validator-236-2.2.0,image,33,media/images/image_129_1586010fb1c66f0bb4d2.png,./downloads/media/images/image_129_1586010fb1c...,1.740385e+09
4,validator-236-2.2.0,image,64,media/images/image_129_1586010fb1c66f0bb4d2.png,./downloads/media/images/image_129_1586010fb1c...,1.740385e+09



Media counts by type:


modality
video    54
image    43
Name: count, dtype: int64

In [17]:
# Add local filepaths of downloaded media to predictions df
if download_df is not None:
    print("\nMerging predictions and download filepaths...")
    merged_results = merge_performance_and_downloads(
        predictions_df=perf_results['predictions'],
        download_df=download_df
    )
    downloaded_count = len(merged_results[merged_results['local_filepath'] != 'not downloaded'])
    print(f"\nTotal predictions with downloaded media: {downloaded_count}")
else:
    merged_results = perf_results
    print("\nNo media downloads - using original performance results")

merged_results


Merging predictions and download filepaths...

Total predictions with downloaded media: 82


,modality,uid,prediction,label,wandb_filepath,validator_run,timestamp,local_filepath
0,video,64,"[0, 1, 0]",1,No Media Found,validator-236-2.2.0,1.740384e+09,not downloaded
1,video,33,"[0.9990643605240621, 0.0009356394759379328, 0]",0,media/videos/video_121_e0858f29644baad423da.gif,validator-236-2.2.0,1.740385e+09,./downloads/media/videos/video_121_e0858f29644...
2,video,33,"[0.9124413390178233, 0.08755866098217666, 0]",0,media/videos/video_124_f9a1d871060237216156.gif,validator-236-2.2.0,1.740385e+09,./downloads/media/videos/video_124_f9a1d871060...
3,video,64,"[0, 1, 0]",1,media/videos/video_128_2dcdc5b28395c9cc09b3.gif,validator-236-2.2.0,1.740385e+09,./downloads/media/videos/video_128_2dcdc5b2839...
4,image,33,"[0.3816100060939789, 0.5700175762176514, 0.048...",0,media/images/image_129_1586010fb1c66f0bb4d2.png,validator-236-2.2.0,1.740385e+09,./downloads/media/images/image_129_1586010fb1c...
...,...,...,...,...,...,...,...,...
269,video,33,"[0.989730260707438, 0.010269739292562008, 0]",0,No Media Found,validator-236-2.2.0,1.740451e+09,not downloaded
270,image,64,"[1, 0, 0]",0,media/images/image_136_99f2b946319523cf9e4c.png,validator-236-2.2.0,1.740451e+09,not downloaded
271,video,64,"[0, 1, 0]",1,media/videos/video_138_d1d58c1e8910162158b1.gif,validator-236-2.2.0,1.740451e+09,not downloaded
272,video,64,"[0, 1, 0]",1,media/videos/video_139_0672852ea3192c779840.gif,validator-236-2.2.0,1.740451e+09,not downloaded


In [18]:
# Save the predictions dataframe
predictions_csv = "miner_predictions.csv"
merged_results.to_csv(predictions_csv, index=False)
print(f"\nSaved predictions to {predictions_csv}")

# Save the performance metrics dataframe
performance_csv = "miner_performance.csv"
performance_df.to_csv(performance_csv, index=False)
print(f"Saved performance metrics to {performance_csv}")


Saved predictions to miner_predictions.csv
Saved performance metrics to miner_performance.csv


In [19]:
from media_gallery import generate_media_gallery

# Generate a basic gallery
generate_media_gallery(
    df=merged_results,
    output_path="media_gallery.html", 
    title="Challenge Media Gallery",
    max_items=100
)

print("Open the generated html in your browser to view media gallery")

Grouped 82 rows into 74 unique media items
Gallery generated successfully at: media_gallery.html
Open the generated html in your browser to view media gallery
